<a href="https://colab.research.google.com/github/MayankKhoria2007/codeX-2.0ai-ml/blob/main/moviercommendusingtfidf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import ast

# ============================================================
# 1️⃣ Load dataset
# ============================================================

movies = pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv')
credits = pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv')

# Merge on title
movies = movies.merge(credits, on='title')


#  Keep only relevant columns

movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

# Fill missing overview
movies['overview'] = movies['overview'].fillna('')

# Parse json like text safely

def convert(text):
    try:
        L = []
        for i in ast.literal_eval(text):
            L.append(i['name'])
        return L
    except:
        return []

# Apply to each relevant feature
for feature in ['genres', 'keywords', 'cast', 'crew']:
    movies[feature] = movies[feature].apply(convert)

#  Convert lists to space-separated strings

for feature in ['genres', 'keywords', 'cast', 'crew']:
    movies[feature] = movies[feature].apply(lambda x: ' '.join(x))

# ============================================================
#  Combine text features into one column
# ============================================================

movies['tags'] = (
    movies['overview'] + ' ' +
    movies['genres'] + ' ' +
    movies['keywords'] + ' ' +
    movies['cast'] + ' ' +
    movies['crew']
)

# Keep final dataframe
new_df = movies[['movie_id', 'title', 'tags']]

# ============================================================
#  TF-IDF Vectorization
# ============================================================

vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
vectors = vectorizer.fit_transform(new_df['tags']).toarray()

# Cosine similarity matrix
similarity = cosine_similarity(vectors)

# ============================================================
#  Recommendation Function
# ============================================================

def recommend(movie):
    movie = movie.lower()
    if movie not in new_df['title'].str.lower().values:
        print("❌ Movie not found in dataset!")
        return

    idx = new_df[new_df['title'].str.lower() == movie].index[0]
    distances = list(enumerate(similarity[idx]))
    movies_list = sorted(distances, reverse=True, key=lambda x: x[1])[1:6]

    print(f"\n🎥 Because you liked '{new_df.iloc[idx].title}', you might also like:\n")
    for i in movies_list:
        print(f" {new_df.iloc[i[0]].title}")

for i in range(5):
    a=input("enter which movie you like")
    recommend(a)

